In [1]:
# Import packages

import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
import pyodbc

In [2]:
# Retrieve keys

from api_keys import server_name, database_name, username, password

In [7]:
# Read data
## Create connection string

connection_string = "DRIVER={ODBC Driver 18 for SQL Server};SERVER={"+ server_name + "};DATABASE={" + database_name + "};UID={" + username + "};PWD={" + password + "}"

## Try to establish a connection

try:
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    print("Connected to Azure SQL Database")

except Exception as e:
    print("Connection failed:", e)

## Save data in DF

queryRent = f"SELECT * FROM rawPropertyRentData"
queryPurchase = f"SELECT * FROM rawPropertyPurchaseData"

rawPropertyRentData_df = pd.read_sql(queryRent, connection)
rawPropertyPurchaseData_df = pd.read_sql(queryPurchase, connection)

Connected to Azure SQL Database


C:\Users\Brecht Nys\AppData\Local\Temp\ipykernel_92\2541375540.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rawPropertyRentData_df = pd.read_sql(queryRent, connection)
C:\Users\Brecht Nys\AppData\Local\Temp\ipykernel_92\2541375540.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rawPropertyPurchaseData_df = pd.read_sql(queryPurchase, connection)


In [8]:
# Clean Database

table_names = ["enrichedPropertyData", "comparisonPropertyData", "historicalPropertyData"]

for table_name in table_names:
    try:
        truncate_query = f"TRUNCATE TABLE {table_name}"
        cursor.execute(truncate_query)
        connection.commit()

        print(f"Table '{table_name}' cleaned.")

    except Exception as e:
        print("Error cleaning table: {table_name}:", e)

Table 'enrichedPropertyData' cleaned.
Table 'comparisonPropertyData' cleaned.
Error cleaning table: {table_name}: ('42S02', '[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot find the object "historicalPropertyData" because it does not exist or you do not have permissions. (4701) (SQLExecDirectW)')


In [9]:
# Create distance definition

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in meters
    R = 6371
    
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

In [10]:
# Calculate Predicted Monthly Rent

for index, row in rawPropertyPurchaseData_df.iterrows():
    
    # Retrieve values purchase property

    property_code = row["propertyCode"]
    property_type = row["propertyType"]
    description = row["description"]
    address = row["address"]
    municipality = row["municipality"]
    country = row["country"]
    lat_purchase = row["latitude"]
    lon_purchase = row["longitude"]
    bathrooms = row["bathrooms"]
    bedrooms = row["bedrooms"]
    size =row["size"]
    status = row["status"]
    floor = row["floor"]
    has_lift = row["hasLift"]
    new_development = row["newDevelopment"]
    new_property = row["newProperty"]
    epc = row["epc"]
    purchase_price = row["purchasePrice"]
    url = row["url"]
    image = row["image"]


    # Run through rent properties
    ## Filter on similar properties
    lower_limit_seize = size - 5
    upper_limit_seize = size + 5

    filtered_rent_property_df = rawPropertyRentData_df.loc[(rawPropertyRentData_df["propertyType"] == property_type) & 
                                                           (rawPropertyRentData_df["bathrooms"] == bathrooms) &
                                                           (rawPropertyRentData_df["bedrooms"] == bedrooms) & 
                                                           (rawPropertyRentData_df["epc"] == epc) & 
                                                           (rawPropertyRentData_df["size"] >= lower_limit_seize) & 
                                                           (rawPropertyRentData_df["size"] <= upper_limit_seize), :]
    filtered_rent_property_df.reset_index(drop=True, inplace = True)


    ## Filter on acceptable distance from purchase property
    ### Calculate distance

    distance_list = []

    for index, row in filtered_rent_property_df.iterrows():

        lat_rent = row["latitude"]
        lon_rent = row["longitude"]

        distance = calculate_distance(lat_purchase, lon_purchase, lat_rent, lon_rent)
        distance_list.append(distance)
    
    filtered_rent_property_df["Distance from property (km)"] = distance_list
    

    ### Filter on acceptable radius

    nearby_rent_property_df = filtered_rent_property_df.loc[filtered_rent_property_df["Distance from property (km)"] <= 0.75, :]

    if len(nearby_rent_property_df["propertyCode"]) <= 2:
        nearby_rent_property_df = filtered_rent_property_df.loc[filtered_rent_property_df["Distance from property (km)"] <= 1, :]
        if len(nearby_rent_property_df["propertyCode"]) <= 2:
            nearby_rent_property_df = filtered_rent_property_df.loc[filtered_rent_property_df["Distance from property (km)"] <= 1.5, :]
            if len(nearby_rent_property_df["propertyCode"]) <= 2:
                nearby_rent_property_df = filtered_rent_property_df.loc[filtered_rent_property_df["Distance from property (km)"] <= 2, :]
                if len(nearby_rent_property_df["propertyCode"]) <= 2:
                    predicted_monthly_rent = 0
                else:
                    predicted_monthly_rent = nearby_rent_property_df["monthlyRent"].mean()
            else:
                predicted_monthly_rent = nearby_rent_property_df["monthlyRent"].mean()
        else:
            predicted_monthly_rent = nearby_rent_property_df["monthlyRent"].mean()
    else:
        predicted_monthly_rent = nearby_rent_property_df["monthlyRent"].mean()

    nearby_rent_property_df.reset_index(drop=True, inplace = True)


    # Save Data to enrichedPropertyData

    sql_query = f"""
        INSERT INTO enrichedPropertyData 
        (propertyCode, propertyType, description, address, municipality, country, latitude, longitude, bathrooms, bedrooms, size, status, floor, hasLift, newDevelopment, newProperty, epc, purchasePrice, predictedMonthlyRent, url, image)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    if cursor.execute(f"SELECT 1 FROM enrichedPropertyData WHERE propertyCode = ?", (property_code,)).fetchone():
        pass
    else:
        cursor.execute(sql_query, (
            property_code, property_type , description, address, municipality, country,
            lat_purchase, lon_purchase, bathrooms, bedrooms, size, status, floor, int(has_lift),
            int(new_development), int(new_property), epc, purchase_price, predicted_monthly_rent, url, image))
        connection.commit()


    # Save Data to comparisonPropertyData

    if len(nearby_rent_property_df["propertyCode"]) > 0:

        for index, row in nearby_rent_property_df.iterrows():

            sql_query = f"""
                INSERT INTO comparisonPropertyData 
                (propertyCodeMain, propertyCodeComp, propertyType, address, municipality, country, latitude, longitude, bathrooms, bedrooms, size, status, floor, hasLift, newDevelopment, newProperty, epc, monthlyRent, url, image)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            cursor.execute(sql_query, (
                property_code, row["propertyCode"], row["propertyType"], row["address"], row["municipality"], row["country"],
                row["latitude"], row["longitude"], row["bathrooms"], row["bedrooms"], row["size"], row["status"], row["floor"], int(row["hasLift"]),
                int(row["newDevelopment"]), int(row["newProperty"]), row["epc"], row["monthlyRent"], row["url"], row["image"]))
            connection.commit()
    else:
        pass


C:\Users\Brecht Nys\AppData\Local\Temp\ipykernel_92\1785286732.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rent_property_df["Distance from property (km)"] = distance_list
C:\Users\Brecht Nys\AppData\Local\Temp\ipykernel_92\1785286732.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rent_property_df["Distance from property (km)"] = distance_list
C:\Users\Brecht Nys\AppData\Local\Temp\ipykernel_92\1785286732.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [11]:
# Save Historical Rent Data

today_date = datetime.today().strftime("%Y-%m-%d")
cities = rawPropertyRentData_df["municipality"].unique()

for x in range(len(cities)):

    city = cities[x]
    attribute = "Monthly LT Rent"

    relevant_data_df = rawPropertyRentData_df.loc[rawPropertyRentData_df["municipality"] == city, :]
    relevant_data_df.reset_index(drop=True, inplace = True)

    ## Number of bedrooms 

    zero_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 0, "monthlyRent"].mean()
    one_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 1, "monthlyRent"].mean() 
    two_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 2, "monthlyRent"].mean()
    three_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 3, "monthlyRent"].mean()
    four_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 4, "monthlyRent"].mean()
    five_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 5, "monthlyRent"].mean()
    more_than_five_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] > 5, "monthlyRent"].mean()


    ## Size of property

    size_one = relevant_data_df.loc[(relevant_data_df["size"] >= 0) & (relevant_data_df["size"] <= 25), "monthlyRent"].mean()
    size_two = relevant_data_df.loc[(relevant_data_df["size"] >= 26) & (relevant_data_df["size"] <= 50), "monthlyRent"].mean()
    size_three = relevant_data_df.loc[(relevant_data_df["size"] >= 51) & (relevant_data_df["size"] <= 75), "monthlyRent"].mean()
    size_four = relevant_data_df.loc[(relevant_data_df["size"] >= 76) & (relevant_data_df["size"] <= 100), "monthlyRent"].mean()
    size_five = relevant_data_df.loc[(relevant_data_df["size"] >= 101) & (relevant_data_df["size"] <= 150), "monthlyRent"].mean()
    size_six = relevant_data_df.loc[(relevant_data_df["size"] >= 151) & (relevant_data_df["size"] <= 200), "monthlyRent"].mean()
    size_seven = relevant_data_df.loc[relevant_data_df["size"] > 200, "monthlyRent"].mean()

    # Clean data

    attribute_values = [zero_bedroom, one_bedroom, two_bedroom, three_bedroom, four_bedroom, five_bedroom, more_than_five_bedroom,
                        size_one, size_two, size_three, size_four, size_five, size_six, size_seven]
    
    attribute_values_cleaned = [0 if pd.isna(x) else x for x in attribute_values]

    # Save Data to enrichedPropertyData

    sql_query = f"""
        INSERT INTO historicalRentData 
        (city, attribute, date, zeroBedroom, oneBedroom, twoBedroom, threeBedroom, fourBedroom, fiveBedroom, moreThanFiveBedroom, twentyFive, fifty, seventyFive, hundred, hundredFifty, twoHundred, moreThanTwoHundred)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    if cursor.execute(
        f"SELECT 1 FROM historicalRentData WHERE city = ? AND attribute = ? AND date = ?",
        (city, attribute, today_date)).fetchone():
        pass
    else:
        cursor.execute(sql_query, (
            city, attribute, today_date, *attribute_values_cleaned))
        connection.commit()


In [12]:
# Save Historical Purchase Data

today_date = datetime.today().strftime("%Y-%m-%d")
cities = rawPropertyPurchaseData_df["municipality"].unique()

for x in range(len(cities)):

    city = cities[x]
    attribute = "Purchase Price"

    relevant_data_df = rawPropertyPurchaseData_df.loc[rawPropertyPurchaseData_df["municipality"] == city, :]
    relevant_data_df.reset_index(drop=True, inplace = True)

    ## Number of bedrooms 

    zero_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 0, "purchasePrice"].mean()
    one_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 1, "purchasePrice"].mean() 
    two_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 2, "purchasePrice"].mean()
    three_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 3, "purchasePrice"].mean()
    four_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 4, "purchasePrice"].mean()
    five_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] == 5, "purchasePrice"].mean()
    more_than_five_bedroom = relevant_data_df.loc[relevant_data_df["bedrooms"] > 5, "purchasePrice"].mean()


    ## Size of property

    size_one = relevant_data_df.loc[(relevant_data_df["size"] >= 0) & (relevant_data_df["size"] <= 25), "purchasePrice"].mean()
    size_two = relevant_data_df.loc[(relevant_data_df["size"] >= 26) & (relevant_data_df["size"] <= 50), "purchasePrice"].mean()
    size_three = relevant_data_df.loc[(relevant_data_df["size"] >= 51) & (relevant_data_df["size"] <= 75), "purchasePrice"].mean()
    size_four = relevant_data_df.loc[(relevant_data_df["size"] >= 76) & (relevant_data_df["size"] <= 100), "purchasePrice"].mean()
    size_five = relevant_data_df.loc[(relevant_data_df["size"] >= 101) & (relevant_data_df["size"] <= 150), "purchasePrice"].mean()
    size_six = relevant_data_df.loc[(relevant_data_df["size"] >= 151) & (relevant_data_df["size"] <= 200), "purchasePrice"].mean()
    size_seven = relevant_data_df.loc[relevant_data_df["size"] > 200, "purchasePrice"].mean()

    # Clean data

    attribute_values = [zero_bedroom, one_bedroom, two_bedroom, three_bedroom, four_bedroom, five_bedroom, more_than_five_bedroom,
                        size_one, size_two, size_three, size_four, size_five, size_six, size_seven]
    
    attribute_values_cleaned = [0 if pd.isna(x) else x for x in attribute_values]

    # Save Data to enrichedPropertyData

    sql_query = f"""
        INSERT INTO historicalPurchaseData 
        (city, attribute, date, zeroBedroom, oneBedroom, twoBedroom, threeBedroom, fourBedroom, fiveBedroom, moreThanFiveBedroom, twentyFive, fifty, seventyFive, hundred, hundredFifty, twoHundred, moreThanTwoHundred)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    if cursor.execute(
        f"SELECT 1 FROM historicalPurchaseData WHERE city = ? AND attribute = ? AND date = ?",
        (city, attribute, today_date)).fetchone():
        pass
    else:
        cursor.execute(sql_query, (
            city, attribute, today_date, *attribute_values_cleaned))
        connection.commit()